En esta celda importamos todas las bibliotecas necesarias para el análisis y modelado. Incluimos bibliotecas para manipulación de datos como pandas y numpy, herramientas de machine learning como scikit-learn, xgboost y otras utilidades como matplotlib para visualización.

In [1]:
# Importar las bibliotecas necesarias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import joblib


Leemos tres conjuntos de datos distintos que contienen información sobre inmuebles en Madrid. Cada archivo contiene datos específicos (como ubicación, características generales y detalles del precio).

Eliminamos columnas innecesarias. Es común que al guardar y cargar archivos CSV se cree una columna adicional (Unnamed: 0) que no aporta información útil. Además, se eliminan columnas duplicadas de latitud y longitud de uno de los datasets.

Combinamos los tres conjuntos de datos utilizando id como clave común. Eliminamos las columnas duplicadas creadas durante el proceso de unión y guardamos el dataset unificado en un nuevo archivo.

In [2]:
lat_lon_df = pd.read_csv('../data/lat_lon_houses_Madrid_cleaned.csv')
houses_clean_df = pd.read_csv('../data/madrid_houses_clean.csv')
houses_df = pd.read_csv('../data/houses_Madrid.csv')

if 'Unnamed: 0' in houses_clean_df.columns:
    houses_clean_df = houses_clean_df.drop('Unnamed: 0', axis=1)

# Para latitude y longitude, usaremos solo los datos de lat_lon_df
# Eliminamos estas columnas de los otros datasets
if 'latitude' in houses_df.columns:
    houses_df = houses_df.drop(['latitude', 'longitude'], axis=1)

merged_df = pd.merge(lat_lon_df, houses_clean_df, on='id', how='inner')

# Luego unimos con houses_df, si hay columnas con el mismo nombre, se le añade como sufijo 'drop' 
final_df = pd.merge(merged_df, houses_df, on='id', how='left', suffixes=('', '_drop'))

final_df = final_df.loc[:, ~final_df.columns.str.endswith('_drop')]

print("Columnas finales:", final_df.columns.tolist())
print("Número de filas:", len(final_df))

final_df.to_csv('../data/unified_houses_madrid.csv', index=False)

Columnas finales: ['id', 'latitude', 'longitude', 'address', 'sq_mt_built', 'n_rooms', 'n_bathrooms', 'n_floors', 'sq_mt_allotment', 'floor', 'buy_price', 'is_renewal_needed', 'has_lift', 'is_exterior', 'energy_certificate', 'has_parking', 'neighborhood', 'district', 'house_type', 'Unnamed: 0', 'title', 'subtitle', 'sq_mt_useful', 'raw_address', 'is_exact_address_hidden', 'street_name', 'street_number', 'portal', 'is_floor_under', 'door', 'neighborhood_id', 'operation', 'rent_price', 'rent_price_by_area', 'is_rent_price_known', 'buy_price_by_area', 'is_buy_price_known', 'house_type_id', 'is_new_development', 'built_year', 'has_central_heating', 'has_individual_heating', 'are_pets_allowed', 'has_ac', 'has_fitted_wardrobes', 'has_garden', 'has_pool', 'has_terrace', 'has_balcony', 'has_storage_room', 'is_furnished', 'is_kitchen_equipped', 'is_accessible', 'has_green_zones', 'has_private_parking', 'has_public_parking', 'is_parking_included_in_price', 'parking_price', 'is_orientation_north'

Cargamos el dataset unificado para revisar las primeras filas y el tipo de datos en cada columna, asegurándonos de que la unión y el guardado se hicieron correctamente.

In [3]:
df = pd.read_csv('../data/unified_houses_madrid.csv')

print(df.head())
print(df.info())

      id   latitude  longitude  \
0  21742  40.344512  -3.689441   
1  21741  40.353850  -3.698362   
2  21738  40.353380  -3.690115   
3  21733  40.353235  -3.689915   
4  21730  40.361389  -3.689422   

                                             address  sq_mt_built  n_rooms  \
0        Calle de Godella, 64, San Cristóbal, Madrid         64.0        2   
1  Calle de la del Manojo de Rosas, Los Ángeles, ...         70.0        3   
2  Carretera de Villaverde a Vallecas, Los Rosale...        108.0        2   
3       Calle de Martinez Oviol, Los Rosales, Madrid         85.0        2   
4    Calle de la Unanimidad, 67, Los Rosales, Madrid        123.0        3   

   n_bathrooms  n_floors  sq_mt_allotment  floor  ...  is_accessible  \
0            1         1              0.0      3  ...            NaN   
1            1         1              0.0      4  ...            NaN   
2            2         1              0.0      4  ...            NaN   
3            1         1              

Separamos las variables predictoras (X) y la variable objetivo (y). También revisamos si existen valores nulos en las variables predictoras para manejarlos más adelante.

In [4]:

df = pd.read_csv('../data/unified_houses_madrid.csv')

ids = df['id'].copy()


X = df.drop(['buy_price'], axis=1)
y = df['buy_price']

X_train = X.drop(['id'], axis=1)

print("Columnas con valores nulos:")
print(X.isnull().sum()[X.isnull().sum() > 0])


Columnas con valores nulos:
sq_mt_useful                    3658
street_name                       99
street_number                   3393
portal                          6735
is_floor_under                   157
door                            6735
rent_price_by_area              6735
house_type_id                    168
is_new_development               230
built_year                      3489
has_central_heating             2228
has_individual_heating          2228
are_pets_allowed                6735
has_ac                          3272
has_fitted_wardrobes            2361
has_garden                      6534
has_pool                        5576
has_terrace                     3889
has_balcony                     5538
has_storage_room                4679
is_furnished                    6735
is_kitchen_equipped             6735
is_accessible                   5246
has_green_zones                 5404
has_private_parking             6735
has_public_parking              6735
is_parking

Identificamos y clasificamos las columnas según el tipo de información que contienen: numéricas, categóricas y binarias. Esto es crucial para definir estrategias de preprocesamiento específicas.

In [5]:
# 1. Primero, veamos qué columnas tenemos en el DataFrame
print("Columnas disponibles en el DataFrame:")
print(df.columns.tolist())

# 2. Ajustamos las listas de columnas según las que realmente existen
columnas_numericas = [
    'sq_mt_built', 'sq_mt_useful', 'n_rooms', 'n_bathrooms', 
    'floor', 'built_year', 'buy_price_by_area',
    'latitude', 'longitude'
]

columnas_categoricas = [
    'house_type', 'energy_certificate', 'district', 'neighborhood'
]

columnas_binarias = [
    'has_lift', 'is_exterior', 'has_parking', 'is_new_development',
    'has_central_heating', 'has_individual_heating', 'has_ac',
    'has_garden', 'has_pool', 'has_terrace', 'has_storage_room',
    'is_furnished', 'is_orientation_north', 'is_orientation_south',
    'is_orientation_east', 'is_orientation_west'
]

# 3. Verificar que todas las columnas existen
print("\nVerificando existencia de columnas:")
for col in columnas_numericas + columnas_categoricas + columnas_binarias:
    if col in df.columns:
        print(f"{col}: ✓")
    else:
        print(f"{col}: ✗")

# 4. Preparar X e y
X = df[columnas_numericas + columnas_categoricas + columnas_binarias]
y = df['buy_price']

# 5. Verificar valores nulos antes de la transformación
print("\nValores nulos por columna:")
print(X.isnull().sum())

Columnas disponibles en el DataFrame:
['id', 'latitude', 'longitude', 'address', 'sq_mt_built', 'n_rooms', 'n_bathrooms', 'n_floors', 'sq_mt_allotment', 'floor', 'buy_price', 'is_renewal_needed', 'has_lift', 'is_exterior', 'energy_certificate', 'has_parking', 'neighborhood', 'district', 'house_type', 'Unnamed: 0', 'title', 'subtitle', 'sq_mt_useful', 'raw_address', 'is_exact_address_hidden', 'street_name', 'street_number', 'portal', 'is_floor_under', 'door', 'neighborhood_id', 'operation', 'rent_price', 'rent_price_by_area', 'is_rent_price_known', 'buy_price_by_area', 'is_buy_price_known', 'house_type_id', 'is_new_development', 'built_year', 'has_central_heating', 'has_individual_heating', 'are_pets_allowed', 'has_ac', 'has_fitted_wardrobes', 'has_garden', 'has_pool', 'has_terrace', 'has_balcony', 'has_storage_room', 'is_furnished', 'is_kitchen_equipped', 'is_accessible', 'has_green_zones', 'has_private_parking', 'has_public_parking', 'is_parking_included_in_price', 'parking_price', 'i

Las variables binarias se transforman en valores numéricos (1.0 y 0.0) utilizando numpy. A continuación, se manejan los valores nulos en las variables numéricas y categóricas, aplicando distintas estrategias de imputación según el tipo de variable: para variables numéricas, se rellenan con la mediana, media o moda según corresponda, y en el caso de las variables categóricas, se rellenan con la moda de cada barrio o un valor predeterminado para columnas específicas como energy_certificate.

Después, se asegura que las columnas tengan los tipos de datos correctos (float64 para variables numéricas y string para las categóricas). El código también verifica si aún existen valores nulos, los tipos de datos y la distribución de las variables binarias.

Finalmente, se comprueba que los valores en las variables binarias estén dentro del rango esperado (0 y 1). Todo esto garantiza que los datos estén listos para ser utilizados en un modelo de machine learning, con valores limpios y en el formato adecuado.

In [6]:
# 1. Crear una copia del DataFrame para evitar warnings
X = df[columnas_numericas + columnas_categoricas + columnas_binarias].copy()
y = df['buy_price'].copy()

# 2. Convertir variables binarias usando numpy
for col in columnas_binarias:
    if col in X.columns:
        # Convertir a array numpy, procesar y volver a pandas
        valores = np.where(X[col].isin([True, 'True', '1', 1]), 1.0, 0.0)
        X[col] = pd.Series(valores, index=X.index, dtype='float64')

# 3. Manejo de valores nulos para variables numéricas
for col in columnas_numericas:
    if col in X.columns:
        if col == 'built_year':
            X[col] = X[col].fillna(X[col].median())
        elif col in ['buy_price_by_area', 'sq_mt_built', 'sq_mt_useful']:
            X[col] = X[col].fillna(X[col].mean())
        elif col in ['n_rooms', 'n_bathrooms', 'floor']:
            X[col] = X[col].fillna(X[col].mode().iloc[0])
        elif col in ['latitude', 'longitude']:
            medianas_barrio = X.groupby('neighborhood')[col].transform('median')
            X[col] = X[col].fillna(medianas_barrio)

# 4. Manejo de valores nulos para variables categóricas
for col in columnas_categoricas:
    if col in X.columns:
        if col == 'energy_certificate':
            X[col] = X[col].fillna('NO_DISPONIBLE')
        else:
            modas_barrio = X.groupby('neighborhood')[col].transform(lambda x: x.mode().iloc[0])
            X[col] = X[col].fillna(modas_barrio)

# 5. Asegurar tipos de datos correctos
X[columnas_numericas] = X[columnas_numericas].astype('float64')
X[columnas_categoricas] = X[columnas_categoricas].astype('string')

# 6. Verificar el resultado
print("Valores nulos después del procesamiento:")
print(X.isnull().sum()[X.isnull().sum() > 0])

print("\nTipos de datos después del procesamiento:")
print(X.dtypes)

# 7. Verificar valores únicos en variables binarias
print("\nDistribución de variables binarias:")
for col in columnas_binarias:
    counts = X[col].value_counts().sort_index()
    total = len(X)
    print(f"\n{col}:")
    for valor, cantidad in counts.items():
        porcentaje = (cantidad/total) * 100
        print(f"{valor:.1f}: {cantidad} ({porcentaje:.1f}%)")

# 8. Verificar que los valores son correctos
print("\nVerificación de rangos:")
for col in columnas_binarias:
    min_val = X[col].min()
    max_val = X[col].max()
    print(f"{col}: min={min_val}, max={max_val}")

Valores nulos después del procesamiento:
Series([], dtype: int64)

Tipos de datos después del procesamiento:
sq_mt_built                      float64
sq_mt_useful                     float64
n_rooms                          float64
n_bathrooms                      float64
floor                            float64
built_year                       float64
buy_price_by_area                float64
latitude                         float64
longitude                        float64
house_type                string[python]
energy_certificate        string[python]
district                  string[python]
neighborhood              string[python]
has_lift                         float64
is_exterior                      float64
has_parking                      float64
is_new_development               float64
has_central_heating              float64
has_individual_heating           float64
has_ac                           float64
has_garden                       float64
has_pool                      

Este código define un conjunto de transformadores para preprocesar diferentes tipos de variables. Las variables numéricas se estandarizan con StandardScaler, las categóricas se convierten en variables binarias mediante OneHotEncoder, y las binarias se transforman en valores numéricos (0.0 y 1.0) usando una función personalizada. Finalmente, se agrupan estos transformadores en un ColumnTransformer que aplica cada uno al tipo de columna correspondiente en el DataFrame.

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.preprocessing import FunctionTransformer

# Definir una función normal para la transformación binaria
def convert_to_float(x):
    return x.astype(float)

# Numéricas: Estandarización
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# Categóricas: One-Hot Encoding
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'))
])

# Variables binarias: Convertir a números
binary_transformer = Pipeline(steps=[
    ('bool_to_int', FunctionTransformer(convert_to_float))
])

# Crear el preprocesador completo
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, columnas_numericas),
        ('cat', categorical_transformer, columnas_categoricas),
        ('bin', binary_transformer, columnas_binarias)
    ])

Este código define una función llamada evaluar_modelo, que evalúa el rendimiento de un modelo de regresión. Toma tres parámetros: y_true (valores reales), y_pred (valores predichos) y nombre_modelo (nombre del modelo para mostrar en los resultados). La función calcula tres métricas de evaluación comunes: MAE (Error absoluto medio), RMSE (Raíz del error cuadrático medio) y R2 (Coeficiente de determinación). Luego, imprime estos resultados y devuelve un diccionario con los valores de estas métricas para su posterior análisis.

In [8]:
# 1. Definir función de evaluación
def evaluar_modelo(y_true, y_pred, nombre_modelo):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    
    print(f"\nResultados para {nombre_modelo}:")
    print(f"MAE: {mae:,.2f} €")
    print(f"RMSE: {rmse:,.2f} €")
    print(f"R2 Score: {r2:.4f}")
    
    return {'mae': mae, 'rmse': rmse, 'r2': r2}

Este código realiza una serie de pasos para procesar y entrenar un modelo de regresión Random Forest en un conjunto de datos. Primero, analiza la distribución de muestras por barrio y agrupa aquellos con menos de 5 muestras en una categoría "OTROS". Luego, divide los datos en conjuntos de entrenamiento y prueba, asegurándose de que ambas particiones contengan las mismas categorías de variables categóricas.

A continuación, transforma los datos utilizando un preprocesador (que incluye normalización de variables numéricas, One-Hot Encoding de variables categóricas y conversión de variables binarias), y entrena un modelo Random Forest usando una búsqueda en cuadrícula (GridSearchCV) para optimizar los hiperparámetros como el número de estimadores, la profundidad máxima y el número mínimo de muestras para dividir.

Finalmente, se entrena el modelo con los mejores parámetros encontrados, se realiza la predicción sobre el conjunto de prueba y se evalúa el rendimiento del modelo utilizando métricas como MAE, RMSE y R² mediante la función evaluar_modelo.

In [9]:
# 1. Analizar la distribución de muestras por barrio
print("Distribución de muestras por barrio:")
neighborhood_counts = X['neighborhood'].value_counts()
print(neighborhood_counts)

print("\nBarrios con pocas muestras (<=5):")
print(neighborhood_counts[neighborhood_counts <= 5])

# 2. Usar train_test_split normal pero con shuffle
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42,
    shuffle=True
)

# 3. Verificar la distribución en train y test
print("\nVerificando categorías únicas por variable categórica:")
for col in columnas_categoricas:
    train_categories = set(X_train[col].unique())
    test_categories = set(X_test[col].unique())
    diff_categories = test_categories - train_categories
    if len(diff_categories) > 0:
        print(f"\n{col}:")
        print(f"Categorías solo en test: {diff_categories}")

# 4. Continuar con el preprocesamiento y entrenamiento
preprocessor.fit(X_train)
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

# 5. Configurar y entrenar Random Forest
rf_param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20],
    'min_samples_split': [2, 5]
}

rf_model = RandomForestRegressor(random_state=42)
rf_grid_search = GridSearchCV(
    estimator=rf_model, 
    param_grid=rf_param_grid, 
    cv=3, 
    n_jobs=1,  # Disable parallel processing to avoid pickling issues
    verbose=1
)

# 6. Entrenar el modelo
rf_grid_search.fit(X_train_transformed, y_train)

# 7. Evaluar el modelo
print(f"\nMejores parámetros para Random Forest: {rf_grid_search.best_params_}")
y_pred = rf_grid_search.predict(X_test_transformed)
resultados = evaluar_modelo(y_test, y_pred, "Random Forest")

Distribución de muestras por barrio:
neighborhood
22    160
24    144
23    143
30    133
89    124
     ... 
21      3
79      2
65      1
55      1
11      1
Name: count, Length: 114, dtype: Int64

Barrios con pocas muestras (<=5):
neighborhood
134    5
80     5
63     5
48     4
21     3
79     2
65     1
55     1
11     1
Name: count, dtype: Int64

Verificando categorías únicas por variable categórica:

neighborhood:
Categorías solo en test: {'65'}
Fitting 3 folds for each of 8 candidates, totalling 24 fits


c:\Users\HP\miniconda3\envs\TFG-IDEALISTA\lib\site-packages\sklearn\preprocessing\_encoders.py:202: UserWarning: Found unknown categories in columns [3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(



Mejores parámetros para Random Forest: {'max_depth': 20, 'min_samples_split': 2, 'n_estimators': 200}

Resultados para Random Forest:
MAE: 10,509.34 €
RMSE: 55,545.44 €
R2 Score: 0.9884


In [10]:
# 1. Entrenar múltiples modelos para comparar
from sklearn.model_selection import KFold, RandomizedSearchCV, cross_val_score


def entrenar_y_evaluar_modelos(X_train, X_test, y_train, y_test):
    modelos = {
        'Random Forest': RandomForestRegressor(
            n_estimators=200,
            max_depth=20,
            min_samples_split=2,
            random_state=42
        ),
        'XGBoost': XGBRegressor(
            n_estimators=200,
            max_depth=7,
            learning_rate=0.1,
            random_state=42
        )
    }

     # Parámetros para búsqueda aleatoria de Random Forest
    param_dist_rf = {
        'n_estimators': [100, 200, 300],
        'max_depth': [10, 20, 30, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }

    # Parámetros para búsqueda aleatoria de XGBoost
    param_dist_xgb = {
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 5, 7, 10],
        'learning_rate': [0.01, 0.1, 0.2],
        'subsample': [0.7, 0.8, 1.0]
    }

    resultados = {}

    for nombre, modelo in modelos.items():
        print(f"\nEntrenando {nombre}...")

        # Dependiendo del modelo, definimos la búsqueda aleatoria
        if nombre == 'Random Forest':
            random_search = RandomizedSearchCV(
                estimator=modelo,
                param_distributions=param_dist_rf,
                n_iter=10,
                cv=5,
                scoring='neg_mean_absolute_error',
                random_state=42,
                n_jobs=-1
            )
        elif nombre == 'XGBoost':
            random_search = RandomizedSearchCV(
                estimator=modelo,
                param_distributions=param_dist_xgb,
                n_iter=10,
                cv=5,
                scoring='neg_mean_absolute_error',
                random_state=42,
                n_jobs=-1
            )

             # Entrenar el modelo con la búsqueda aleatoria
        random_search.fit(X_train, y_train)

        # Obtener el mejor modelo de la búsqueda aleatoria
        best_model = random_search.best_estimator_

        # Validación cruzada para evaluar el modelo
        kf = KFold(n_splits=5, shuffle=True, random_state=42)
        scores = cross_val_score(best_model, X_train, y_train, cv=kf, scoring='neg_mean_absolute_error')

        print(f"\n{nombre} - Mejor modelo: {random_search.best_params_}")
        print(f"{nombre} - MAE promedio de validación cruzada: {-scores.mean()}")

        # Entrenar el modelo con los mejores parámetros y predecir sobre el conjunto de test
        y_pred = best_model.predict(X_test)

        # Evaluar el modelo final en el conjunto de test
        resultados[nombre] = evaluar_modelo(y_test, y_pred, nombre)

    return modelos, resultados


In [11]:
# 2. Analizar importancia de características
def analizar_importancia_features(modelo, feature_names):
    if hasattr(modelo, 'feature_importances_'):
        importances = modelo.feature_importances_
        indices = np.argsort(importances)[::-1]

        print("\nImportancia de características:")
        for f in range(len(feature_names)):
            print("%d. %s (%f)" % (f + 1, feature_names[indices[f]], importances[indices[f]]))


# 3. Obtener nombres de características después del preprocesamiento
def get_feature_names(preprocessor, columnas_numericas, columnas_categoricas, columnas_binarias):
    numeric_features = columnas_numericas
    categorical_features = []
    for i, col in enumerate(columnas_categoricas):
        cats = preprocessor.named_transformers_['cat'].named_steps['onehot'].categories_[i][1:]
        categorical_features.extend([f"{col}_{cat}" for cat in cats])
    binary_features = columnas_binarias
    return numeric_features + categorical_features + binary_features

# 4. Entrenar y evaluar modelos
modelos, resultados = entrenar_y_evaluar_modelos(X_train_transformed, X_test_transformed, y_train, y_test)

# 5. Obtener nombres de características y analizar importancia
feature_names = get_feature_names(preprocessor, columnas_numericas, columnas_categoricas, columnas_binarias)
analizar_importancia_features(modelos['XGBoost'], feature_names)



Entrenando Random Forest...

Random Forest - Mejor modelo: {'n_estimators': 300, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_depth': None}
Random Forest - MAE promedio de validación cruzada: 13081.342889575306

Resultados para Random Forest:
MAE: 10,237.11 €
RMSE: 54,325.80 €
R2 Score: 0.9889

Entrenando XGBoost...

XGBoost - Mejor modelo: {'subsample': 0.8, 'n_estimators': 200, 'max_depth': 10, 'learning_rate': 0.1}
XGBoost - MAE promedio de validación cruzada: 11462.812168423332

Resultados para XGBoost:
MAE: 9,333.08 €
RMSE: 37,060.43 €
R2 Score: 0.9948


In [12]:
def guardar_mejor_modelo(modelos, resultados, ruta_modelo='../data/models/mejor_modelo.joblib', ruta_preprocessor='../data/models/preprocessor.joblib'):
    """
    Compara los modelos basándose en R2 score y guarda el mejor.
    
    Args:
        modelos (dict): Diccionario con los modelos entrenados
        resultados (dict): Diccionario con las métricas de cada modelo
        ruta_modelo (str): Ruta donde guardar el mejor modelo
        ruta_preprocessor (str): Ruta donde guardar el preprocessor
    
    Returns:
        tuple: (nombre_mejor_modelo, r2_score)
    """
    # Encontrar el modelo con mejor R2 score
    mejor_r2 = -float('inf')
    mejor_modelo_nombre = None
    
    for nombre, metricas in resultados.items():
        r2_actual = metricas['r2']
        print(f"Modelo: {nombre}, R2 Score: {r2_actual:.4f}")
        
        if r2_actual > mejor_r2:
            mejor_r2 = r2_actual
            mejor_modelo_nombre = nombre
    
    print(f"\nMejor modelo: {mejor_modelo_nombre} con R2 Score: {mejor_r2:.4f}")
    
    # Guardar el mejor modelo y el preprocessor
    mejor_modelo = modelos[mejor_modelo_nombre]
    joblib.dump(mejor_modelo, ruta_modelo)
    joblib.dump(preprocessor, ruta_preprocessor)
    
    print(f"Modelo guardado en: {ruta_modelo}")
    print(f"Preprocessor guardado en: {ruta_preprocessor}")
    
    return mejor_modelo_nombre, mejor_r2

# Uso de la función:
mejor_modelo_nombre, mejor_r2 = guardar_mejor_modelo(modelos, resultados)

Modelo: Random Forest, R2 Score: 0.9889
Modelo: XGBoost, R2 Score: 0.9948

Mejor modelo: XGBoost con R2 Score: 0.9948
Modelo guardado en: ../data/models/mejor_modelo.joblib
Preprocessor guardado en: ../data/models/preprocessor.joblib


In [14]:
# === DEFINIR MODELOS PARA ENTRENAMIENTO FINAL ===
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

# Definir modelos con los mejores parámetros encontrados
xgb_model = XGBRegressor(
    n_estimators=200,
    max_depth=10,
    learning_rate=0.1,
    random_state=42
)

rf_model = RandomForestRegressor(
    n_estimators=200,
    max_depth=10,
    random_state=42
)

print("✅ Modelos definidos:")
print(f"XGBoost: {type(xgb_model)}")
print(f"Random Forest: {type(rf_model)}")

# Verificar que el mejor modelo está identificado
print(f"\nMejor modelo identificado: {mejor_modelo_nombre}")
print(f"Mejor R2 Score: {mejor_r2:.4f}")

✅ Modelos definidos:
XGBoost: <class 'xgboost.sklearn.XGBRegressor'>
Random Forest: <class 'sklearn.ensemble._forest.RandomForestRegressor'>

Mejor modelo identificado: XGBoost
Mejor R2 Score: 0.9948


In [15]:
# === VERIFICACIÓN Y ENTRENAMIENTO FINAL ===
print("=== VERIFICACIÓN DEL MODELO ===\n")

# Verificar estado de los modelos definidos anteriormente
for nombre, modelo in [("Random Forest", rf_model), ("XGBoost", xgb_model)]:
    print(f"Modelo: {nombre}")
    print(f"Tipo: {type(modelo)}")
    print(f"¿Tiene método predict?: {hasattr(modelo, 'predict')}")
    
    if hasattr(modelo, 'n_estimators'):
        print(f"N estimators: {modelo.n_estimators}")
    
    # Probar predicción
    try:
        test_pred = modelo.predict(X_test_transformed[:1])
        print("✅ Predicción exitosa")
    except Exception as e:
        print(f"❌ Error en predicción: {e}")
    print()

# Encontrar el mejor modelo
print("=== ENTRENAMIENTO DEL MEJOR MODELO ===")

# Determinar cuál fue el mejor modelo
if 'XGBoost' in mejor_modelo_nombre:
    mejor_modelo = xgb_model
    print("Mejor modelo seleccionado: XGBoost")
else:
    mejor_modelo = rf_model
    print("Mejor modelo seleccionado: Random Forest")

print(f"Tipo del mejor modelo: {type(mejor_modelo)}")

# Entrenar el modelo con todos los datos
print("Entrenando el modelo...")
mejor_modelo.fit(X_train_transformed, y_train)
print("✅ Modelo entrenado exitosamente")

# Verificar que funciona después del entrenamiento
try:
    test_pred = mejor_modelo.predict(X_test_transformed[:1])
    print("✅ Predicción de prueba exitosa")
except Exception as e:
    print(f"❌ Error en predicción después del entrenamiento: {e}")

print(f"\n=== GUARDANDO MODELO ===")
print(f"R2 Score del mejor modelo: {mejor_r2:.4f}")

# Paths absolutos
import os
base_path = r"c:\Users\HP\Desktop\tfg-alvaro-carrera-idealista\backend"
model_path = os.path.join(base_path, "data", "models", "mejor_modelo.joblib")
preprocessor_path = os.path.join(base_path, "data", "models", "preprocessor.joblib")

print(f"Guardando modelo en: {model_path}")
print(f"Guardando preprocessor en: {preprocessor_path}")

# Guardar
joblib.dump(mejor_modelo, model_path)
joblib.dump(preprocessor, preprocessor_path)
print("✅ Modelo y preprocessor guardados correctamente")

print(f"\n=== VERIFICACIÓN DE CARGA ===")
# Cargar para verificar
modelo_cargado = joblib.load(model_path)
preprocessor_cargado = joblib.load(preprocessor_path)

print(f"Modelo cargado - Tipo: {type(modelo_cargado)}")
print(f"¿Tiene método predict?: {hasattr(modelo_cargado, 'predict')}")

# Probar predicción con modelo cargado
try:
    pred_test = modelo_cargado.predict(X_test_transformed[:1])
    print("✅ Predicción con modelo cargado exitosa")
    print(f"Predicción de prueba: {pred_test[0]:.2f}")
except Exception as e:
    print(f"❌ Error al cargar modelo: {e}")

print(f"\n=== RESUMEN ===")
print(f"Mejor modelo: {mejor_modelo_nombre}")
print(f"R2 Score: {mejor_r2:.4f}")
print(f"El modelo está listo para usar en el backend")

=== VERIFICACIÓN DEL MODELO ===

Modelo: Random Forest
Tipo: <class 'sklearn.ensemble._forest.RandomForestRegressor'>
¿Tiene método predict?: True
N estimators: 200
❌ Error en predicción: This RandomForestRegressor instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

Modelo: XGBoost
Tipo: <class 'xgboost.sklearn.XGBRegressor'>
¿Tiene método predict?: True
N estimators: 200
❌ Error en predicción: need to call fit or load_model beforehand

=== ENTRENAMIENTO DEL MEJOR MODELO ===
Mejor modelo seleccionado: XGBoost
Tipo del mejor modelo: <class 'xgboost.sklearn.XGBRegressor'>
Entrenando el modelo...
✅ Modelo entrenado exitosamente
✅ Predicción de prueba exitosa

=== GUARDANDO MODELO ===
R2 Score del mejor modelo: 0.9948
Guardando modelo en: c:\Users\HP\Desktop\tfg-alvaro-carrera-idealista\backend\data\models\mejor_modelo.joblib
Guardando preprocessor en: c:\Users\HP\Desktop\tfg-alvaro-carrera-idealista\backend\data\models\preprocessor.joblib
✅ Mode

In [18]:
# === RECREAR PREPROCESSOR CON TRANSFORMADORES ESTÁNDAR ===
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline

print("=== RECREANDO PREPROCESSOR ===")

# Definir las columnas usando los nombres reales del dataset
columnas_numericas = ['sq_mt_built', 'sq_mt_useful', 'n_rooms', 'n_bathrooms', 
                     'floor', 'built_year', 'buy_price_by_area', 'latitude', 'longitude']
columnas_binarias = ['has_lift', 'is_exterior', 'has_parking', 'is_new_development',
                    'has_central_heating', 'has_individual_heating', 'has_ac',
                    'has_garden', 'has_pool', 'has_terrace', 'has_storage_room',
                    'is_furnished', 'is_orientation_north', 'is_orientation_south',
                    'is_orientation_east', 'is_orientation_west']
columnas_categoricas = ['house_type', 'energy_certificate', 'district', 'neighborhood']

print(f"Columnas numéricas: {len(columnas_numericas)}")
print(f"Columnas binarias: {len(columnas_binarias)}")
print(f"Columnas categóricas: {len(columnas_categoricas)}")

# Verificar que todas las columnas existen
todas_columnas = columnas_numericas + columnas_binarias + columnas_categoricas
columnas_disponibles = set(X_train.columns)
columnas_faltantes = set(todas_columnas) - columnas_disponibles
if columnas_faltantes:
    print(f"⚠️ Columnas faltantes: {columnas_faltantes}")
else:
    print("✅ Todas las columnas están disponibles")

# Transformadores usando solo funciones estándar de sklearn
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# Para las columnas binarias, simplemente las dejamos como están (ya son 0/1)
binary_transformer = Pipeline(steps=[
    ('passthrough', 'passthrough')  # No transformación
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(drop='first', handle_unknown='ignore'))
])

# Crear el preprocessor estándar
preprocessor_standard = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, columnas_numericas),
        ('bin', binary_transformer, columnas_binarias),
        ('cat', categorical_transformer, columnas_categoricas)
    ],
    remainder='drop'
)

print("✅ Preprocessor estándar creado")

# Entrenar el preprocessor con los datos de entrenamiento
print("Entrenando preprocessor...")
preprocessor_standard.fit(X_train)
print("✅ Preprocessor entrenado")

# Transformar los datos
X_train_transformed_new = preprocessor_standard.transform(X_train)
X_test_transformed_new = preprocessor_standard.transform(X_test)

print(f"Forma de datos transformados - Train: {X_train_transformed_new.shape}")
print(f"Forma de datos transformados - Test: {X_test_transformed_new.shape}")

# Verificar que funciona
print("Verificando compatibilidad...")
test_sample = X_train.iloc[:1]
test_transformed = preprocessor_standard.transform(test_sample)
print(f"Muestra de prueba transformada: {test_transformed.shape}")
print("✅ Preprocessor funcionando correctamente")

=== RECREANDO PREPROCESSOR ===
Columnas numéricas: 9
Columnas binarias: 16
Columnas categóricas: 4
✅ Todas las columnas están disponibles
✅ Preprocessor estándar creado
Entrenando preprocessor...
✅ Preprocessor entrenado
Forma de datos transformados - Train: (5388, 167)
Forma de datos transformados - Test: (1347, 167)
Verificando compatibilidad...
Muestra de prueba transformada: (1, 167)
✅ Preprocessor funcionando correctamente


c:\Users\HP\miniconda3\envs\TFG-IDEALISTA\lib\site-packages\sklearn\preprocessing\_encoders.py:202: UserWarning: Found unknown categories in columns [3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [17]:
# === VERIFICAR ESTRUCTURA DE DATOS ===
print("=== VERIFICACIÓN DE DATOS ===")
print(f"Forma de X_train: {X_train.shape}")
print(f"Columnas disponibles en X_train:")
print(list(X_train.columns))
print(f"\nTipo de X_train: {type(X_train)}")

# También verificar y_train
print(f"\nForma de y_train: {y_train.shape}")
print(f"Tipo de y_train: {type(y_train)}")

# Mostrar una muestra de los datos
print(f"\nPrimeras 3 filas de X_train:")
print(X_train.head(3))

=== VERIFICACIÓN DE DATOS ===
Forma de X_train: (5388, 29)
Columnas disponibles en X_train:
['sq_mt_built', 'sq_mt_useful', 'n_rooms', 'n_bathrooms', 'floor', 'built_year', 'buy_price_by_area', 'latitude', 'longitude', 'house_type', 'energy_certificate', 'district', 'neighborhood', 'has_lift', 'is_exterior', 'has_parking', 'is_new_development', 'has_central_heating', 'has_individual_heating', 'has_ac', 'has_garden', 'has_pool', 'has_terrace', 'has_storage_room', 'is_furnished', 'is_orientation_north', 'is_orientation_south', 'is_orientation_east', 'is_orientation_west']

Tipo de X_train: <class 'pandas.core.frame.DataFrame'>

Forma de y_train: (5388,)
Tipo de y_train: <class 'pandas.core.series.Series'>

Primeras 3 filas de X_train:
      sq_mt_built  sq_mt_useful  n_rooms  n_bathrooms  floor  built_year  \
4935         65.0     94.340591      3.0          1.0    3.0      1960.0   
349          85.0     75.000000      3.0          2.0    1.0      1970.0   
2183         75.0     94.3405

In [19]:
# === ENTRENAR MODELO CON NUEVO PREPROCESSOR ===
print("=== ENTRENAMIENTO CON PREPROCESSOR ESTÁNDAR ===")

# Usar el preprocessor estándar que acabamos de crear
X_train_std = preprocessor_standard.transform(X_train)
X_test_std = preprocessor_standard.transform(X_test)

print(f"Datos transformados - Train: {X_train_std.shape}")
print(f"Datos transformados - Test: {X_test_std.shape}")

# Crear un nuevo modelo XGBoost
from xgboost import XGBRegressor
xgb_final = XGBRegressor(
    n_estimators=200,
    max_depth=10,
    learning_rate=0.1,
    random_state=42
)

print("Entrenando modelo XGBoost...")
xgb_final.fit(X_train_std, y_train)
print("✅ Modelo entrenado")

# Evaluar el modelo
from sklearn.metrics import r2_score, mean_squared_error
y_pred_std = xgb_final.predict(X_test_std)
r2_std = r2_score(y_test, y_pred_std)
mse_std = mean_squared_error(y_test, y_pred_std)

print(f"R2 Score con preprocessor estándar: {r2_std:.4f}")
print(f"MSE: {mse_std:.2f}")

# Guardar el modelo y preprocessor estándar
import os
base_path = r"c:\Users\HP\Desktop\tfg-alvaro-carrera-idealista\backend"
model_path = os.path.join(base_path, "data", "models", "mejor_modelo.joblib")
preprocessor_path = os.path.join(base_path, "data", "models", "preprocessor.joblib")

print(f"\n=== GUARDANDO MODELO ESTÁNDAR ===")
print(f"Guardando modelo en: {model_path}")
print(f"Guardando preprocessor en: {preprocessor_path}")

joblib.dump(xgb_final, model_path)
joblib.dump(preprocessor_standard, preprocessor_path)
print("✅ Modelo y preprocessor guardados")

# Verificar carga
print(f"\n=== VERIFICACIÓN DE CARGA ===")
modelo_cargado = joblib.load(model_path)
preprocessor_cargado = joblib.load(preprocessor_path)

print(f"Modelo cargado - Tipo: {type(modelo_cargado)}")
print(f"Preprocessor cargado - Tipo: {type(preprocessor_cargado)}")

# Probar predicción
test_sample = X_test.iloc[:1]
test_transformed = preprocessor_cargado.transform(test_sample)
pred_test = modelo_cargado.predict(test_transformed)

print(f"✅ Predicción exitosa: {pred_test[0]:.2f}")
print("🎉 ¡El modelo está listo para producción!")

=== ENTRENAMIENTO CON PREPROCESSOR ESTÁNDAR ===
Datos transformados - Train: (5388, 167)
Datos transformados - Test: (1347, 167)
Entrenando modelo XGBoost...


c:\Users\HP\miniconda3\envs\TFG-IDEALISTA\lib\site-packages\sklearn\preprocessing\_encoders.py:202: UserWarning: Found unknown categories in columns [3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


✅ Modelo entrenado
R2 Score con preprocessor estándar: 0.9938
MSE: 1642576997.04

=== GUARDANDO MODELO ESTÁNDAR ===
Guardando modelo en: c:\Users\HP\Desktop\tfg-alvaro-carrera-idealista\backend\data\models\mejor_modelo.joblib
Guardando preprocessor en: c:\Users\HP\Desktop\tfg-alvaro-carrera-idealista\backend\data\models\preprocessor.joblib
✅ Modelo y preprocessor guardados

=== VERIFICACIÓN DE CARGA ===
Modelo cargado - Tipo: <class 'xgboost.sklearn.XGBRegressor'>
Preprocessor cargado - Tipo: <class 'sklearn.compose._column_transformer.ColumnTransformer'>
✅ Predicción exitosa: 180958.38
🎉 ¡El modelo está listo para producción!
